# Backpropagation Practice

> #### Tobias Reaper

## *Data Science Unit 4 Sprint 2 Assignment 2*

---
---

Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn. 

In [1]:
# === Imports === #
import pandas as pd
import numpy as np

In [5]:
# Set up the dataset

X = np.array(([0, 0, 1],
              [0, 1, 1],
              [1, 0, 1],
              [0, 1, 0],
              [1, 0, 0],
              [1, 1, 1],
              [0, 0, 0,]), dtype=float)

y = np.array(([0],
              [1],
              [1],
              [1],
              [1],
              [0],
              [0]), dtype=float)

In [15]:
class Multitron:
    def __init__(self):
        # Set up the architecture
        self.inputs = 3
        self.hidden_nodes = 4
        self.output_nodes = 1
        
        # Set up the initial weights - layer 1: 3x4
        self.weights1 = np.random.rand(self.inputs, self.hidden_nodes)
        
        # Hidden to output - layer 2: 4x1
        self.weights2 = np.random.rand(self.hidden_nodes, self.output_nodes)
        
    def sigmoid(self, s):
        return 1 / (1 + np.exp(-s))
    
    def sigmoid_prime(self, s):
        return s * (1 - s)
    
    def feed(self, X):
        """
        Generates predictions.
        Calculates the neural network inference using feed forward.
        """
        # Weighted sum of inputs => hidden layer
        self.hidden_sum = np.dot(X, self.weights1)

        # Activate weighted sum
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        
        # Weighted sum of hidden => output
        self.output_sum = np.dot(self.activated_hidden, self.weights2)
        
        # Final activation of output
        self.activated_output = self.sigmoid(self.output_sum)
        
        return self.activated_output
    
    def back(self, X, y, o):
        """Backwards propagation through the network."""
        
        # Calc error in the output
        self.o_error = y - o
        
        # Apply sigmoid derivative to the error
        # How far off is it in relation to the sigmoid f(x) of the output?
        # or, hidden => output
        self.o_delta = self.o_error * self.sigmoid_prime(o)
        
        # z2 error
        self.z2_error = self.o_delta.dot(self.weights2.T)
        
        # How much "far out" can be explained by the input => hidden
        self.z2_delta = self.z2_error * self.sigmoid_prime(self.activated_hidden)
        
        # Adjustments to first weights (input => hidden)
        self.weights1 += X.T.dot(self.z2_delta)
        # Adjustments to second weights (hidden => output)
        self.weights2 += self.activated_hidden.T.dot(self.o_delta)
        
    def train(self, X, y):
        o = self.feed(X)
        self.back(X, y, o)

In [16]:
# === Train === #
nn = Multitron()

nn.train(X, y)

In [17]:
# === First error === #
nn.o_error

array([[-0.68161163],
       [ 0.27578994],
       [ 0.26822359],
       [ 0.28780451],
       [ 0.28033113],
       [-0.75923541],
       [-0.66545129]])

In [18]:
# === First gradient === #
nn.o_delta

array([[-0.14792146],
       [ 0.05508348],
       [ 0.05264684],
       [ 0.05899217],
       [ 0.05655557],
       [-0.13878596],
       [-0.14814667]])

In [19]:
# === Second error === #
nn.z2_error

array([[-0.01458329, -0.01468032, -0.0353626 , -0.13881992],
       [ 0.00543057,  0.00546671,  0.01316844,  0.05169422],
       [ 0.00519035,  0.00522488,  0.01258593,  0.04940751],
       [ 0.00581592,  0.00585462,  0.01410287,  0.05536241],
       [ 0.0055757 ,  0.0056128 ,  0.01352036,  0.05307573],
       [-0.01368264, -0.01377367, -0.03317863, -0.13024652],
       [-0.0146055 , -0.01470267, -0.03541644, -0.13903128]])

In [20]:
# === Second gradient === #
nn.z2_delta

array([[-0.00297449, -0.00366757, -0.00775182, -0.03469191],
       [ 0.00063091,  0.00135683,  0.00225576,  0.01121895],
       [ 0.00076906,  0.00125746,  0.00208311,  0.00998432],
       [ 0.00117125,  0.00145858,  0.00328436,  0.01218163],
       [ 0.00127834,  0.00136363,  0.0030969 ,  0.01090612],
       [-0.00099457, -0.00322948, -0.00385393, -0.01746862],
       [-0.00365137, -0.00367567, -0.00885411, -0.03475782]])

---
---

## Try building/training a more complex MLP on a bigger dataset.

Use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to build the cannonical handwriting digit recognizer and see what kind of accuracy you can achieve. 

If you need inspiration, the internet is chalk-full of tutorials, but I want you to see how far you can get on your own first. I've linked to the original MNIST dataset above but it will probably be easier to download data through a neural network library. If you reference outside resources make sure you understand every line of code that you're using from other sources, and share with your fellow students helpful resources that you find.


### Parts
1. Gathering & Transforming the Data
2. Making MNIST a Binary Problem
3. Estimating your Neural Network (the part you focus on)

### Gathering the Data 

`keras` has a handy method to pull the mnist dataset for you. You'll notice that each observation is a 28x28 arrary which represents an image. Although most Neural Network frameworks can handle higher dimensional data, that is more overhead than necessary for us. We need to flatten the image to one long row which will be 784 values (28X28). Basically, you will be appending each row to one another to make on really long row. 

In [39]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [2]:
# input image dimensions
img_rows, img_cols = 28, 28

In [31]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [32]:
x_train = x_train.reshape(x_train.shape[0], img_rows * img_cols)
x_test = x_test.reshape(x_test.shape[0], img_rows * img_cols)

# Normalize Our Data
x_train = x_train / 255
x_test = x_test / 255

In [16]:
# Now the data should be in a format you're more familiar with
x_train.shape

(60000, 784)

### Making MNIST a Binary Problem 
MNIST is multiclass classification problem; however we haven't covered all the necessary techniques to handle this yet. You would need to one-hot encode the target, use a different loss metric, and use softmax activations for the last layer. This is all stuff we'll cover later this week, but let us simply the problem for now: Zero or all else.

In [33]:
import numpy as np

y_temp = np.zeros(y_train.shape)
y_temp[np.where(y_train == 0.0)[0]] = 1
y_train = y_temp

y_temp = np.zeros(y_test.shape)
y_temp[np.where(y_test == 0.0)[0]] = 1
y_test = y_temp

In [40]:
# A Nice Binary target for ya to work with
y_train

array([0., 1., 0., ..., 0., 0., 0.])

### Estimating Your `net

In [ ]:
##### Your Code Here #####



---
---

## Stretch Goals: 

- Make MNIST a multiclass problem using cross entropy & soft-max
- Implement Cross Validation model evaluation on your MNIST implementation 
- Research different [Gradient Descent Based Optimizers](https://keras.io/optimizers/)
 - [Siraj Raval the evolution of gradient descent](https://www.youtube.com/watch?v=nhqo0u1a6fw)
- Build a housing price estimation model using a neural network. How does its accuracy compare with the regression models that we fit earlier on in class?

---
---

## Extras, Read All About It ®

In [9]:
# === Make a prediction === #
nn = Multitron()

output = nn.feed(X[0])

print("Input", X[0])
print("Output", output)

Input [0. 0. 1.]
Output [0.75366448]


In [10]:
# === Calculate error === #
error = y[0] - output
error

array([-0.75366448])

In [11]:
# === Do the same for all outputs === #
output_all = nn.feed(X)
error_all = y - output_all

print(output_all)
print(error_all)

[[0.75366448]
 [0.78707117]
 [0.79935951]
 [0.76004871]
 [0.77404792]
 [0.8214482 ]
 [0.71920973]]
[[-0.75366448]
 [ 0.21292883]
 [ 0.20064049]
 [ 0.23995129]
 [ 0.22595208]
 [-0.8214482 ]
 [-0.71920973]]
